In [192]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import re
import requests
import seaborn as sns

from db import create_connection
from db import (
    articles,
    services,
    urls
)

from stop_words_list import stop_words

In [193]:
pd.set_option('max_colwidth', 400)

In [194]:
import os
current_path = os.path.abspath(os.getcwd())
plots_directory = os.path.join(current_path, 'datasets/transcripts')

if not os.path.exists(plots_directory):
    os.makedirs(plots_directory)

In [195]:
connection = create_connection()

In [196]:
df = pd.read_csv('datasets/1_clean_data.csv')
df.head(1)

,Unnamed: 0,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words,koronawirus_anywhere,koronawirus_anywhere_count
0,0,2020-08-30 14:52:00,2020-08,2020-08-30,kaja,Salmonella w kurczakach! Główny Inspektorat Sanitarny ostrzega,https://lublin.se.pl/salmonella-w-kurczakach-glowny-inspektorat-sanitarny-ostrzega-ak-zuvx-nuNm-w7xC.html,2,0,1,0,263,2,1


In [197]:
def url_to_transcript(url):
    page = requests.get(url).text
    bs = BeautifulSoup(page, "lxml")
    container = bs.find(class_="main-content")
    article = container.find("article")
#     article.find("p", {"class": "playerBoard__text playerBoard__text--icon"}).decompose()
#     article.find("p", {"class": "playerBoard__text playerBoard__title"}).decompose()
#     article.find("p", {"class": "playerBoard__text"}).decompose()
    try:
        article.find("p", {"class": "playerBoard__text playerBoard__text--icon"}).decompose()
    except:
        print("NO CLASS: playerBoard__text playerBoard__text--icon")

    try:
        article.find("p", {"class": "playerBoard__text playerBoard__title"}).decompose()
    except:
        print("NO CLASS: playerBoard__text playerBoard__title")

    try:
        article.find("p", {"class": "playerBoard__text"}).decompose()
    except:
        print("NO CLASS: playerBoard__text")
    texts = article.find_all("p")
    articles = []
    for t in texts:
        article = t.text
        articles.append(article)
    return articles

In [198]:
data = {}

In [199]:
names = ["2020-01", "2020-02", "2020-03", "2020-04", "2020-05", "2020-06", "2020-07", "2020-08"]

In [ ]:
for name in names:
    name_urls = df[['url']].where(df['publication_month'] == name).dropna()
    text = []
    for key, url in name_urls.items():
        for k, v in url.items():
            text.append(url_to_transcript(v))
    text_string = ''.join([str(item) for item in text])
    data[name] = text_string

NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: pl

NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: pl

NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: playerBoard__text
NO CLASS: playerBoard__text playerBoard__text--icon
NO CLASS: playerBoard__text playerBoard__title
NO CLASS: pl

In [ ]:
for i, name in enumerate(names):
    with open("datasets/transcripts/" + name + ".txt", "wb") as file:
        pickle.dump(data[name], file)

In [ ]:
data = {}
for i, name in enumerate(names):
    with open("datasets/transcripts/" + name + ".txt", "rb") as file:
        data[name] = pickle.load(file)

In [ ]:
data.keys()

In [ ]:
data['2020-01']

In [ ]:
def combine_text(list_of_text):
    combined_text = ''.join(list_of_text)
    return combined_text

In [ ]:
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [ ]:
data_combined['2020-01']

In [ ]:
pd.set_option('max_colwidth', 150)
data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

In [ ]:
data_df.transcript.loc['2020-02']

In [ ]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    #text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r"\d+", "",  text)
    #text = re.sub('*d', '', text)
    #text = re.sub('"*s"', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

In [ ]:
data_clean.transcript.loc['2020-02']

In [ ]:
next(iter(data.keys()))

In [ ]:
names = list(data.keys())

In [ ]:
data_df['publication_month'] = names

In [ ]:
data_df

In [ ]:
data_df.to_pickle("datasets/2_corpus.pkl")

In [ ]:
# from stop_words import get_stop_words

In [ ]:
# stop_words = get_stop_words('polish')
# stop_words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

In [ ]:
data_dtm.to_pickle("datasets/3_dtm.pkl")

In [ ]:
data_clean.to_pickle('datasets/4_data_clean.pkl')
pickle.dump(cv, open("datasets/5_cv.pkl", "wb"))

In [ ]:
data_dtm.sum()